# 使用自己数据集训练的模型权重文件对所选图像进行超分辨处理

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os

# 忽略TensorFlow的AVX警告
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 定义SRCNN模型
def srcnn_model():
    inputs = tf.keras.layers.Input(shape=(None, None, 3))
    conv1 = tf.keras.layers.Conv2D(64, (9, 9), activation='relu', padding='same')(inputs)
    conv2 = tf.keras.layers.Conv2D(32, (1, 1), activation='relu', padding='same')(conv1)
    outputs = tf.keras.layers.Conv2D(3, (5, 5), padding='same')(conv2)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# 加载SRCNN模型权重
def load_srcnn_weights(model, filename='srcnn.weights.h5'):#自己训练的模型权重文件
    model.load_weights(filename)

# 图像超分辨率处理函数
def super_resolution(model, image):
    # 将图像归一化到0-1之间
    image = image / 255.0
    # 对图像进行超分辨率处理
    sr_image = model.predict(np.expand_dims(image, axis=0))[0]
    # 将图像像素值还原到0-255之间
    sr_image = np.clip(sr_image * 255.0, 0, 255).astype('uint8')
    return sr_image

if __name__ == "__main__":
    # 加载模型
    model = srcnn_model()
    load_srcnn_weights(model)

    # 加载模糊图像
    blurry_image = cv2.imread('img_001.png')#想要进行超分辨处理的图像路径

    # 进行超分辨率处理
    sr_image = super_resolution(model, blurry_image)

    # 显示结果
    cv2.imshow('Blurry Image', blurry_image)#原图
    cv2.imshow('Super-Resolved Image', sr_image)#超分辨处理之后的图像
    cv2.waitKey(0)
    cv2.destroyAllWindows()
